In [14]:
from statsmodels.regression.rolling import RollingOLS # estimates relationship b/w dep & indep var using min square diff over a rolling window
import pandas_datareader.data as web # takes web info and stores as dataframe
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np # calculations
import datetime as dt
import yfinance as yf # stock data
import statsmodels.api as sm # stat models
import pandas_ta # technical indicators calculator
import warnings

warnings.filterwarnings('ignore') # annoying :(

# collect sp500 stock data over the past decade, leaving a time gap for testing
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')
symbols_list = sp500['Symbol'].unique().tolist()

end_date = '2024-02-01'
start_date = pd.to_datetime(end_date) - pd.DateOffset(365*10)

df = yf.download(tickers=symbols_list, start=start_date, end=end_date)
df = df.stack() # multi indexed

df.index.names = ['date', 'ticker']
df.columns = df.columns.str.lower()

# calculate technical indicators
# Garman-Klass Volatility, RSI, Bollinger Bands, ATR, MACD, Dollar Volume

df['garman-klass_vol'] = (0.5 * (np.log(df['high'] / df['low'])**2)) - (2 * np.log(2) - 1) * (np.log(df['adj close'] / df['open'])**2)
df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x : pandas_ta.rsi(close=x, length=20))
df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x : pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x : pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x : pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'], low=stock_data['low'], close=stock_data['close'], length=14)
    return atr.sub(atr.mean()).div(atr.std()) # normalize it

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std()) # normalize it

df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)
df['dollar_vol'] = (df['adj close']*df['volume'])/1e6 # divide by a million

# aggregate to a monthly level and filter top 150 liquid stocks

[*********************100%%**********************]  503 of 503 completed

3 Failed downloads:
['GEV', 'SOLV', 'SW']: YFChartError("%ticker%: Data doesn't exist for startDate = 1391403600, endDate = 1706763600")


In [28]:
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_vol', 'volume', 'open', 'high', 'low', 'close']]
data = (pd.concat([df.unstack('ticker')['dollar_vol'].resample('M').mean().stack('ticker').to_frame('dollar_vol'),
    df.unstack()[last_cols].resample('M').last().stack('ticker')], axis=1)).dropna()

data

dollar_vol   adj close       atr   bb_high    bb_low  \
date       ticker                                                          
2014-03-31 A        126.319499   36.615665 -0.857146  3.678265  3.594556   
           AAL      325.555888   34.504341  0.296276  3.637816  3.534184   
           AAPL    3992.431099   16.861725 -1.043426  2.893676  2.860098   
           ABBV     196.676052   33.445557 -1.171081  3.577047  3.520859   
           ABT      271.044225   31.452080 -0.971604  3.525575  3.466211   
...                        ...         ...       ...       ...       ...   
2024-01-31 ABNB     607.570974  144.139999 -0.809756  5.023789  4.879181   
           CEG      178.015812  121.574860  0.243683  4.819868  4.705704   
           GEHC     195.719215   73.306999 -0.657909  4.368420  4.284151   
           KVUE     337.750731   20.332666 -1.637289  3.114890  3.036682   
           VLTO     106.635028   76.542038 -0.863830  4.384079  4.324689   

                     bb_mid  garman-klass_vol      macd        rsi  
date       ticker                                                   
2014-03-31 A       3.636410         -0.001604 -0.402019  47.426588  
           AAL     3.586000         -0.000081  0.258451  51.337338  
           AAPL    2.876887         -0.006781 -0.216542  57.630168  
           ABBV    3.548953         -0.070839 -0.083475  52.861470  
           ABT     3.495893         -0.015633 -0.194079  50.126821  
...                     ...               ...       ...        ...  
2024-01-31 ABNB    4.951485          0.000177  0.627040  55.591518  
           CEG     4.762786          0.000225  0.130419  58.680206  
           GEHC    4.326285          0.000185 -0.464739  48.485817  
           KVUE    3.075786         -0.000285  0.493494  48.524614  
           VLTO    4.354384          0.000123 -0.520899  48.633597  

[57979 rows x 9 columns]

In [58]:
data['dollar_vol'] = data['dollar_vol'].unstack('ticker').rolling(window=5*12, min_periods=1).mean().stack()
data['dollar_vol_rank'] = data.groupby('date')['dollar_vol'].rank(ascending=False)

data = data[data['dollar_vol_rank'] < 150].drop(['dollar_vol', 'dollar_vol_rank'], axis=1)


KeyError: 'dollar_vol'

In [38]:
def calculate_returns(df):
    outlier_cutoff = 0.005 # 99.5 percentile
    lags = [1, 2, 3, 6, 9, 12]
    
    for lag in lags:
        df[f'return_{lag}m'] = (df['adj close']
                               .pct_change(lag)
                               .pipe(lambda x : x.clip(lower=x.quantile(outlier_cutoff),
                                                        upper=x.quantile(1-outlier_cutoff)))
                               .add(1)
                               .pow(1/lag)
                               .sub(1))
    return df

data = data.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()

data

dollar_vol   adj close       atr   bb_high    bb_low  \
date       ticker                                                          
2015-03-31 A         99.865538   38.491062 -0.994405  3.698666  3.647010   
           AAL      521.629404   50.110546  1.039930  4.012115  3.795229   
           AAPL    5168.957041   27.882063 -0.793663  3.399814  3.345213   
           ABBV     376.697015   39.305679 -0.801353  3.748955  3.637127   
           ABT      194.397831   38.674713 -0.932665  3.712882  3.674980   
...                        ...         ...       ...       ...       ...   
2024-01-31 CARR     206.140129   54.378563 -0.178544  4.058332  4.012046   
           OTIS     170.835090   87.755676 -0.784925  4.501768  4.452281   
           ABNB     878.997795  144.139999 -0.809756  5.023789  4.879181   
           CEG      178.532754  121.574860  0.243683  4.819868  4.705704   
           GEHC     214.046048   73.306999 -0.657909  4.368420  4.284151   

                     bb_mid  garman-klass_vol      macd        rsi  \
date       ticker                                                    
2015-03-31 A       3.672838         -0.001756 -0.093624  53.020375   
           AAL     3.903672         -0.001614  1.173724  54.797238   
           AAPL    3.372514         -0.005673 -0.250246  50.588893   
           ABBV    3.693041         -0.058350 -0.360851  47.251169   
           ABT     3.693931         -0.014045 -0.128544  48.603266   
...                     ...               ...       ...        ...   
2024-01-31 CARR    4.035189          0.000114 -0.304138  47.148919   
           OTIS    4.477024          0.000555  0.133877  54.419267   
           ABNB    4.951485          0.000177  0.627040  55.591518   
           CEG     4.762786          0.000225  0.130419  58.680206   
           GEHC    4.326285          0.000185 -0.464739  48.485817   

                   dollar_vol_rank  return_1m  return_2m  return_3m  \
date       ticker                                                     
2015-03-31 A                 237.0  -0.013212   0.050138   0.006587   
           AAL                32.0   0.101879   0.038059  -0.004640   
           AAPL                1.0  -0.031372   0.032591   0.042114   
           ABBV               49.0  -0.032397  -0.015110  -0.034055   
           ABT               115.0  -0.021955   0.017387   0.011385   
...                            ...        ...        ...        ...   
2024-01-31 CARR              243.0  -0.047694   0.027836   0.048225   
           OTIS              282.0  -0.011512   0.015327   0.047728   
           ABNB               40.0   0.058763   0.068124   0.068101   
           CEG               272.0   0.043716   0.003958   0.026908   
           GEHC              230.0  -0.050842   0.035373   0.033040   

                   return_6m  return_9m  return_12m  
date       ticker                                    
2015-03-31 A        0.004032   0.002050    0.004171  
           AAL      0.069241   0.023940    0.031584  
           AAPL     0.037238   0.034484    0.042802  
           ABBV     0.004732   0.006577    0.013545  
           ABT      0.019948   0.015743    0.017376  
...                      ...        ...         ...  
2024-01-31 CARR    -0.012832   0.032094    0.016748  
           OTIS    -0.003332   0.005380    0.007392  
           ABNB    -0.009017   0.020887    0.021926  
           CEG      0.040448   0.052858    0.031249  
           GEHC    -0.010029  -0.011232    0.004624  

[51992 rows x 16 columns]

In [64]:
factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3',
                             'famafrench',
                             start='2015')[0].drop('RF', axis=1)

factor_data.index = factor_data.index.to_timestamp()

factor_data = factor_data.resample('M').last().div(100)

factor_data.index.name = 'date'

factor_data = factor_data.join(data['return_1m']).sort_index()

factor_data.xs('AAPL', level=1).head()

,Mkt-RF,SMB,HML,RMW,CMA,return_1m
date,,,,,,
2015-03-31,-0.0112,0.0307,-0.0037,0.0009,-0.0052,-0.031372
2015-04-30,0.0059,-0.0309,0.0182,0.0006,-0.0061,0.005786
2015-05-31,0.0136,0.0083,-0.0114,-0.0179,-0.0074,0.045339
2015-06-30,-0.0153,0.0290,-0.0079,0.0044,-0.0158,-0.037228
2015-07-31,0.0154,-0.0455,-0.0413,0.0030,-0.0241,-0.032926


In [65]:
observations = factor_data.groupby(level=1).size()
valid_stocks = observations[observations >= 10]

factor_data = factor_data[factor_data.index.get_level_values('ticker').isin(valid_stocks.index)]

factor_data

Mkt-RF     SMB     HML     RMW     CMA  return_1m
date       ticker                                                   
2015-03-31 AAL    -0.0112  0.0307 -0.0037  0.0009 -0.0052   0.101879
           AAPL   -0.0112  0.0307 -0.0037  0.0009 -0.0052  -0.031372
           ABBV   -0.0112  0.0307 -0.0037  0.0009 -0.0052  -0.032397
           ABT    -0.0112  0.0307 -0.0037  0.0009 -0.0052  -0.021955
           ACN    -0.0112  0.0307 -0.0037  0.0009 -0.0052   0.040653
...                   ...     ...     ...     ...     ...        ...
2024-01-31 WDC     0.0070 -0.0574 -0.0233  0.0069 -0.0099   0.093183
           WFC     0.0070 -0.0574 -0.0233  0.0069 -0.0099   0.019504
           WMT     0.0070 -0.0574 -0.0233  0.0069 -0.0099   0.048208
           WYNN    0.0070 -0.0574 -0.0233  0.0069 -0.0099   0.036439
           XOM     0.0070 -0.0574 -0.0233  0.0069 -0.0099   0.028306

[15905 rows x 6 columns]

In [68]:
betas = factor_data.groupby(level=1,
                            group_keys=False).apply(lambda x : RollingOLS(endog=x['return_1m'],
                                     exog=sm.add_constant(x.drop('return_1m', axis=1)),
                                     window=min(24, x.shape[0]),
                                     min_nobs=len(x.columns)+1)
               .fit(params_only=True)
               .params.drop('const', axis=1))

betas                                                           

Mkt-RF       SMB       HML       RMW       CMA
date       ticker                                                  
2015-03-31 AAL          NaN       NaN       NaN       NaN       NaN
           AAPL         NaN       NaN       NaN       NaN       NaN
           ABBV         NaN       NaN       NaN       NaN       NaN
           ABT          NaN       NaN       NaN       NaN       NaN
           ACN          NaN       NaN       NaN       NaN       NaN
...                     ...       ...       ...       ...       ...
2024-01-31 WDC     1.518886 -0.716663  1.813275 -0.496544 -1.683671
           WFC     1.076844  0.344716  1.622968 -0.206620 -1.309866
           WMT     0.544710  0.230867 -0.911366  0.666801  0.898345
           WYNN    0.977131  1.190642 -0.909066  1.264618  0.756128
           XOM     0.778583  0.036038  0.483825  0.805126  0.585608

[15905 rows x 5 columns]